## I have a shapefile named saline10k_1502 that contains a low, medium, and high calculated estimate CO2 storage potential in saline aquifers in North America, I am only interested in the Canadian CO2 storage, I will create a shapefile containing the total CO2 capacity estimate for a given cell, Because the aquifers are stacked there are aquifers that overlap each other and I am looking for the sum of all the saline aquifer CO2 capacity estimates in each cell.

In [1]:
import os
import shutil
import geopandas as gpd

In [13]:
data = gpd.read_file(r'..\saline10k_1502\saline10k_1502.shp')


In [16]:
print(data.drop('geometry',axis=1).head(3).to_markdown())

|    | COL_ROW   | PARTNERSHI   | ARRA_PROJE   | RESOURCE_N     | BASIN_NAME   |   RSC_AREA_C |   VOL_LOW |   VOL_MED |   VOL_HIGH |   DEPTH_FT |   THICKNESS_ |   SALINITY_T |   PRESSURE_P |   TEMPERATUR |   POROSITY_P |   PERMEABILI |   ASSESSED | CYCLE_OF_L   |   OVERLAP |   DUPLICATE |   MED_CALCED |   Shape_Leng |   Shape_Area |
|---:|:----------|:-------------|:-------------|:---------------|:-------------|-------------:|----------:|----------:|-----------:|-----------:|-------------:|-------------:|-------------:|-------------:|-------------:|-------------:|-----------:|:-------------|----------:|------------:|-------------:|-------------:|-------------:|
|  0 | 314 - 367 | PCOR         |              | Basal Cambrian |              |      1.1e+07 |         0 |         0 |          0 |          0 |            0 |       202396 |         4877 |          290 |    0.037079  |      34      |          1 | Atlas V, v1  |         1 |           0 |            0 |        40000 |        1e+

# The Canadian saline aquifers will under attribute Partnership – PCOR and the seven saline aquifers of RESOURCE_N attribute Basal Cambrian, Beaverhill Lake Group, Elk Point Group, Rundle Group, Viking, Winterburn Group, and Woodbend Group.

In [3]:
canadian_data = data.loc[(data['RESOURCE_N']=='Basal Cambrian')|(data['RESOURCE_N']=='Beaverhill Lake Group')|(data['RESOURCE_N']=='Elk Point Group')|(data['RESOURCE_N']=='Rundle Group')|(data['RESOURCE_N']=='Viking')|(data['RESOURCE_N']=='Winterburn Group')|(data['RESOURCE_N']=='Woodbend Group')|(data['PARTNERSHI']=='PCOR')]

# Create a Shapefile and CSV containing lat longs of total Canadian CO2 capacity estimate I will calculate it  by Summation of VOL_LOW, VOL_MED, VOL_HIGH, and RSC_AREA_C for each Cell or the Cells that overlap in the same location

In [4]:
co2 = canadian_data[['VOL_LOW','VOL_MED','VOL_HIGH','RSC_AREA_C','geometry','RESOURCE_N']]

## Here is I will create a "spatial_id" column to create an ID to cells so the cells overlapped will have one ID (or spatial ID 😉), Then I will implement aggregation with dissolve based on my spatial ID.

In [5]:
co2['spatial_id'] = list(zip(co2['geometry'].centroid.x , co2['geometry'].centroid.y))

D:\anaconda3\envs\PYTHONGIS\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [6]:
sum_co2 = co2.dissolve(by="spatial_id",aggfunc='sum')
print(sum_co2.head(3).to_markdown())

| spatial_id                        | geometry                                                                                                                           |     VOL_LOW |     VOL_MED |    VOL_HIGH |   RSC_AREA_C |
|:----------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|------------:|------------:|------------:|-------------:|
| (-1336251.7400000002, 1363935.68) | POLYGON ((-1341251.74 1358935.68, -1341251.74 1368935.68, -1331251.74 1368935.68, -1331251.74 1358935.68, -1341251.74 1358935.68)) | 0           | 0           | 0           |  1.85008e+06 |
| (-1326251.7400000002, 1333935.68) | POLYGON ((-1331251.74 1328935.68, -1331251.74 1338935.68, -1321251.74 1338935.68, -1321251.74 1328935.68, -1331251.74 1328935.68)) | 3.19865e+06 | 6.05149e+06 | 1.0374e+07  |  1.79924e+07 |
| (-1326251.7400000002, 1343935.68) | POLYGON ((-1331251.74 1338935.68, -1331251.74 1348

## re-indexing & delete spatial _id 
### goodbye spatial _id

In [7]:
sum_co2.reset_index(inplace=True)
sum_co2.drop(columns=['spatial_id'],inplace=True)
print(sum_co2.head(3).to_markdown())

|    | geometry                                                                                                                           |     VOL_LOW |     VOL_MED |    VOL_HIGH |   RSC_AREA_C |
|---:|:-----------------------------------------------------------------------------------------------------------------------------------|------------:|------------:|------------:|-------------:|
|  0 | POLYGON ((-1341251.74 1358935.68, -1341251.74 1368935.68, -1331251.74 1368935.68, -1331251.74 1358935.68, -1341251.74 1358935.68)) | 0           | 0           | 0           |  1.85008e+06 |
|  1 | POLYGON ((-1331251.74 1328935.68, -1331251.74 1338935.68, -1321251.74 1338935.68, -1321251.74 1328935.68, -1331251.74 1328935.68)) | 3.19865e+06 | 6.05149e+06 | 1.0374e+07  |  1.79924e+07 |
|  2 | POLYGON ((-1331251.74 1338935.68, -1331251.74 1348935.68, -1321251.74 1348935.68, -1321251.74 1338935.68, -1331251.74 1338935.68)) | 8.07035e+06 | 1.52682e+07 | 2.61741e+07 |  5.54231e+07 |


## create latitude and longitude columns

In [8]:
sum_co2['lat'] = sum_co2.to_crs(4326)['geometry'].centroid.y
sum_co2['lon'] = sum_co2.to_crs(4326)['geometry'].centroid.x
print(sum_co2.head(3).to_markdown())

D:\anaconda3\envs\PYTHONGIS\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
D:\anaconda3\envs\PYTHONGIS\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


|    | geometry                                                                                                                           |     VOL_LOW |     VOL_MED |    VOL_HIGH |   RSC_AREA_C |     lat |      lon |
|---:|:-----------------------------------------------------------------------------------------------------------------------------------|------------:|------------:|------------:|-------------:|--------:|---------:|
|  0 | POLYGON ((-1341251.74 1358935.68, -1341251.74 1368935.68, -1331251.74 1368935.68, -1331251.74 1358935.68, -1341251.74 1358935.68)) | 0           | 0           | 0           |  1.85008e+06 | 55.5994 | -121.452 |
|  1 | POLYGON ((-1331251.74 1328935.68, -1331251.74 1338935.68, -1321251.74 1338935.68, -1321251.74 1328935.68, -1331251.74 1328935.68)) | 3.19865e+06 | 6.05149e+06 | 1.0374e+07  |  1.79924e+07 | 55.3663 | -121.16  |
|  2 | POLYGON ((-1331251.74 1338935.68, -1331251.74 1348935.68, -1321251.74 1348935.68, -1321251.74 1338935.68, -1331251.74 133

## creating a folder with name 'sum_co2' to contain CO2 capacity estimate files

In [9]:
if os.path.isdir('sum_co2'):
    shutil.rmtree('sum_co2',ignore_errors=True)

os.mkdir('sum_co2')

sum_co2.to_file(r"sum_co2/sum_co2.shp")
sum_co2.to_file(r"sum_co2/sum_co2.csv",driver='CSV')

# Create Shapefiles and CSV containing lat longs of each of the seven Canadian saline aquifers (Seven shapefiles, seven CSV) with Keep all existing attribute values. 

In [10]:
saline_aquifers = data.loc[(data['RESOURCE_N']=='Basal Cambrian')|(data['RESOURCE_N']=='Beaverhill Lake Group')|(data['RESOURCE_N']=='Elk Point Group')|(data['RESOURCE_N']=='Rundle Group')|(data['RESOURCE_N']=='Viking')|(data['RESOURCE_N']=='Winterburn Group')|(data['RESOURCE_N']=='Woodbend Group')]

## creating a folder with name 'saline_aquifers' to contain folders are contain saline aquifers files

In [11]:
if os.path.isdir('saline_aquifers'):
    shutil.rmtree('saline_aquifers',ignore_errors=True)

os.mkdir('saline_aquifers')

In [12]:
for inx,data in saline_aquifers.groupby('RESOURCE_N'):
    data['lat'] = data.to_crs(4326)['geometry'].centroid.y
    data['lon'] = data.to_crs(4326)['geometry'].centroid.x
    os.mkdir(os.path.join('saline_aquifers',str(inx)))
    data.to_file(os.path.join('saline_aquifers',str(inx),str(inx+'.shp')))
    data.to_file(os.path.join('saline_aquifers',str(inx),str(inx+'.csv')),driver='CSV')

D:\anaconda3\envs\PYTHONGIS\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
D:\anaconda3\envs\PYTHONGIS\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


Basal Cambrian is done
Beaverhill Lake Group is done
Elk Point Group is done
Rundle Group is done
Viking is done
Winterburn Group is done
Woodbend Group is done
